In [ ]:
# datasets.py
# This script contains the dataset class and data loading functionality.

import h5py  # Library for handling HDF5 files
import pandas as pd  # DataFrame handling
import numpy as np  # Numerical operations
import torch  # PyTorch for tensor operations and neural networks
from torch.utils.data import Dataset, DataLoader  # Dataset and DataLoader classes from PyTorch
from torchvision import transforms  # Transformations for image data
from PIL import Image  # Image handling from the Python Imaging Library (PIL)
import io  # Input/output library for byte handling

# Define a custom dataset class for handling skin cancer data
class SkinCancerDataset(Dataset):
    def __init__(self, metadata_file, hdf5_file, transform=None):
        self.metadata = pd.read_csv(metadata_file)  # Load metadata CSV file
        self.hdf5_file = h5py.File(hdf5_file, 'r')  # Open the HDF5 file containing images
        self.transform = transform  # Store the transformation to be applied on images

    def __len__(self):
        return len(self.metadata)  # Return the number of samples

    def __getitem__(self, idx):
        image_id = self.metadata.iloc[idx]['isic_id']  # Get image ID from metadata
        label = self.metadata.iloc[idx]['target']  # Get label (target) from metadata

        # Check if the image ID is in the HDF5 file
        if image_id in self.hdf5_file:
            image = self.hdf5_file[image_id][()]  # Load the image from HDF5 file

            if isinstance(image, np.bytes_):
                # If image data is in bytes format, decode it
                image = np.frombuffer(image, dtype=np.uint8)
                image = Image.open(io.BytesIO(image))  # Open the image from bytes
            elif isinstance(image, np.ndarray):
                # If image data is a numpy array, convert it to PIL Image
                image = Image.fromarray(image)
            else:
                raise TypeError(f"Unexpected data format for image ID {image_id}. Expected numpy array or bytes, got {type(image)}.")

            image = image.convert("RGB")  # Ensure the image is in RGB format

            if self.transform:
                image = self.transform(image)  # Apply transformations if specified

            return image, torch.tensor(label, dtype=torch.float32)  # Return the transformed image and label as a tensor
        else:
            raise KeyError(f"Image ID {image_id} not found in HDF5 file.")  # Raise an error if the image is not found

    def __del__(self):
        self.hdf5_file.close()  # Ensure the HDF5 file is closed when the dataset object is deleted

# Function to get a DataLoader for the dataset
def get_dataloader(metadata_file, hdf5_file, batch_size, mean, std, augment=False):
    if augment:
        transform = transforms.Compose([
            transforms.RandomResizedCrop(224),  # Randomly crop and resize images
            transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
            transforms.ToTensor(),  # Convert images to PyTorch tensors
            transforms.Normalize(mean=mean, std=std),  # Normalize images with mean and std deviation
        ])
    else:
        transform = transforms.Compose([
            transforms.Resize(256),  # Resize images
            transforms.CenterCrop(224),  # Center crop images
            transforms.ToTensor(),  # Convert images to PyTorch tensors
            transforms.Normalize(mean=mean, std=std),  # Normalize images with mean and std deviation
        ])
    
    dataset = SkinCancerDataset(metadata_file, hdf5_file, transform)  # Create the dataset
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)  # Create the DataLoader
    
    return dataloader



: 

In [ ]:
# Model.py
# This script defines the model architecture.

import torch
import torch.nn as nn  # Import neural network module from PyTorch
import torchvision.models as models  # Import pre-trained models from torchvision

# Define a model class for skin cancer detection
class SkinCancerModel(nn.Module):
    def __init__(self, model_name='efficientnet_v2_m', pretrained=True):
        super(SkinCancerModel, self).__init__()
        
        # Load a pre-trained EfficientNetV2 model
        if model_name == 'efficientnet_v2_m':
            self.backbone = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.IMAGENET1K_V1 if pretrained else None)
        elif model_name == 'efficientnet_v2_s':
            self.backbone = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.IMAGENET1K_V1 if pretrained else None)
        else:
            raise ValueError(f"Model {model_name} is not supported")  # Raise an error if the model name is not supported
        
        # Modify the classifier to fit our binary classification task
        num_features = self.backbone.classifier[1].in_features  # Get the number of input features to the classifier
        self.backbone.classifier = nn.Sequential(
            nn.Linear(num_features, 1)  # Replace the classifier with a single linear layer for binary classification
        )
    
    def forward(self, x):
        return self.backbone(x)  # Define the forward pass of the model



: 

In [ ]:
# utils.py
import torch
import torch.optim as optim  # Import optimization algorithms
import torch.nn as nn  # Import neural network module
from tqdm import tqdm  # Progress bar
from torch.cuda.amp import GradScaler, autocast  # Mixed precision training tools

# Function to train the model
def train_model(model, train_loader, test_loader, num_epochs, learning_rate, save_path, device, accumulation_steps=4):
    model.to(device)  # Move the model to the specified device (GPU or CPU)
    
    criterion = nn.BCEWithLogitsLoss()  # Define the loss function (binary cross-entropy with logits)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Define the optimizer (Adam)
    scaler = GradScaler()  # Initialize the GradScaler for mixed precision
    
    best_val_loss = float('inf')  # Initialize the best validation loss with infinity
    
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        
        # Iterate over the training data
        for step, (images, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)):
            images, targets = images.to(device), targets.to(device)  # Move images and targets to the device
            
            with autocast():  # Enable mixed precision
                outputs = model(images)  # Forward pass
                loss = criterion(outputs, targets.unsqueeze(1))  # Compute the loss
                loss = loss / accumulation_steps  # Normalize the loss
            
            scaler.scale(loss).backward()  # Scale the gradients and perform backward pass
            
            if (step + 1) % accumulation_steps == 0:
                scaler.step(optimizer)  # Update the weights
                scaler.update()  # Update the scaler
                optimizer.zero_grad()  # Zero the gradients
            
            running_loss += loss.item() * images.size(0)  # Accumulate the running loss
        
        torch.cuda.empty_cache()  # Clean up GPU memory

        epoch_loss = running_loss / len(train_loader.dataset)  # Compute the epoch loss
        
        val_loss = validate_model(model, test_loader, criterion, device)  # Validate the model
        
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f} - Val Loss: {val_loss:.4f}")
        
        # Save the model if the validation loss improves
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            save_model(model, save_path)

# Function to validate the model
def validate_model(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    
    with torch.no_grad():  # Disable gradient computation
        for images, targets in test_loader:
            images, targets = images.to(device), targets.to(device)  # Move images and targets to the device
            
            with autocast():  # Enable mixed precision
                outputs = model(images)  # Forward pass
                loss = criterion(outputs, targets.unsqueeze(1))  # Compute the loss
            
            running_loss += loss.item() * images.size(0)  # Accumulate the running loss
    
    val_loss = running_loss / len(test_loader.dataset)  # Compute the validation loss
    return val_loss

# Function to save the model
def save_model(model, save_path):
    torch.save(model.state_dict(), save_path)  # Save the model's state dictionary to the specified path
    print(f"Model saved to {save_path}")  # Print a message indicating that the model was saved



: 

In [ ]:
# inference.py

import pandas as pd  # Import pandas for data handling
import torch  # Import PyTorch
from torchvision import transforms, models  # Import transformations and models
from torch.utils.data import DataLoader  # Import DataLoader class
from datasets import SkinCancerDataset  # Import the custom dataset class
from model import SkinCancerModel  # Import the model class

# Define the test function
def test(model_path, test_metadata_file, test_hdf5_file, batch_size, mean, std, device):
    # Initialize the transformations
    transform = transforms.Compose([
        transforms.Resize(256),  # Resize the images
        transforms.CenterCrop(224),  # Center crop the images
        transforms.ToTensor(),  # Convert the images to tensors
        transforms.Normalize(mean=mean, std=std)  # Normalize the images
    ])
    
    # Load the test dataset
    test_dataset = SkinCancerDataset(test_metadata_file, test_hdf5_file, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    # Initialize the model and load the trained weights
    model = SkinCancerModel()
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    
    model.eval()  # Set the model to evaluation mode
    predictions = []
    
    with torch.no_grad():  # Disable gradient computation
        for images, _ in test_loader:
            images = images.to(device)  # Move images to the device
            outputs = model(images)  # Get the model predictions
            predictions.append(outputs.cpu().numpy())  # Store the predictions

    predictions = np.concatenate(predictions)  # Concatenate predictions
    return predictions



: 

In [ ]:
# train.py
import torch
from torchvision import models
from datasets import get_dataloader
from utils import train_model

# Set device to 'cuda' (GPU) if available, otherwise fall back to 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define the mean and standard deviation for image normalization
# These values are commonly used for pre-trained models on ImageNet
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Set training parameters
model_name = 'efficientnet_v2_m'  # Name of the model architecture to use
num_epochs = 1  # Number of epochs to train the model
batch_size = 2  # Number of samples processed per batch
save_path = 'models/model.pth'  # File path where the trained model will be saved

# Paths to the metadata and HDF5 files for training and testing datasets
train_metadata_file = 'data/train-metadata.csv'  # Metadata file for training data
train_hdf5_file = 'data/train-image.hdf5'  # HDF5 file containing training images
test_metadata_file = 'data/test-metadata.csv'  # Metadata file for testing data
test_hdf5_file = 'data/test-image.hdf5'  # HDF5 file containing testing images

# Load the training data using the get_dataloader function
# This function will return a DataLoader object that handles batching and shuffling
train_loader = get_dataloader(
    metadata_file=train_metadata_file,  # Path to training metadata file
    hdf5_file=train_hdf5_file,  # Path to HDF5 file with training images
    batch_size=batch_size,  # Batch size for the DataLoader
    mean=mean,  # Mean values for normalization
    std=std,  # Standard deviation values for normalization
    augment=True  # Apply data augmentation during training
)

# Load the testing data using the get_dataloader function
# This DataLoader will be used for evaluating the model after each epoch
test_loader = get_dataloader(
    metadata_file=test_metadata_file,  # Path to testing metadata file
    hdf5_file=test_hdf5_file,  # Path to HDF5 file with testing images
    batch_size=batch_size,  # Batch size for the DataLoader
    mean=mean,  # Mean values for normalization
    std=std,  # Standard deviation values for normalization
    augment=False  # Do not apply data augmentation during testing
)

# Load the EfficientNet-V2 model with pre-trained weights
model = models.efficientnet_v2_m(weights='DEFAULT')

# Modify the output layer of the model to match our binary classification task
# The original model has an output layer for 1000 classes, so we replace it with a single output unit
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 1)

# Train the model using the train_model function
# This function will handle the training loop, including forward pass, backward pass, and weight updates
train_model(
    model=model,  # The model to train
    train_loader=train_loader,  # DataLoader for training data
    test_loader=test_loader,  # DataLoader for testing data
    num_epochs=num_epochs,  # Number of epochs to train
    learning_rate=0.001,  # Learning rate for the optimizer
    save_path=save_path,  # Path to save the trained model
    device=device  # Device to use for training ('cuda' or 'cpu')
)


: 